In [1]:
import pandas as pd
import numpy as np
import config
data = pd.read_parquet('../data/parquet/final.parquet')

In [8]:
percentage_partyline = {}

for party in ["AfD"]:
    party_votes = data[data[f"{party}_ground_truth"].notna()]
    partyline_correct = (party_votes[f"{party}_ground_truth"] == party_votes[f"{party}_prediction"]).sum() / len(party_votes)
    percentage_partyline[party] = round(partyline_correct * 100)

percentage_partyline

{'AfD': 70}

In [10]:
(party_votes[f"{party}_ground_truth"] == party_votes[f"{party}_prediction"]).sum()

np.int64(266)

In [3]:
import plotly.graph_objects as go

percentage_partyline = {
    'AfD': 55,
    'DIE_GRÜNEN': 58,
    'DIE_LINKE': 59,
    'FDP': 65,
    'SPD': 70,
    'Union': 67
}

colors = {
    'AfD': '#009EE0',
    'DIE_GRÜNEN': '#00A65A',
    'DIE_LINKE': '#BE3075',
    'FDP': '#FFD700',
    'SPD': '#E3000F',
    'Union': '#000000'
}

fig = go.Figure(go.Bar(
    x=list(percentage_partyline.keys()),
    y=list(percentage_partyline.values()),
    marker_color=[colors[party] for party in percentage_partyline],
    width=0.4
))

fig.update_layout(
    title='Percentage of Party-Line Agreement',
    xaxis_title='Political Party',
    yaxis_title='Agreement (%)',
    yaxis=dict(range=[0, 100]),
    template='plotly_dark'
)

fig.update_traces(text=list(percentage_partyline.values()), textposition='auto')

fig.show()


In [4]:
percentage_partyline_by_category = {}

for party in config.PARTIES:
    party_votes = data[data[f"{party}_ground_truth"].notna()].copy()
    party_votes["partyline_correct"] = (party_votes[f"{party}_ground_truth"] == party_votes[f"{party}_prediction"])
    categories_vc = party_votes["category"].value_counts()
    relevant_categories = categories_vc[categories_vc > 20].index.tolist()
    party_votes = party_votes[party_votes["category"].isin(relevant_categories)]
    party_votes.groupby("category")["partyline_correct"].mean().reset_index()
    percentage_partyline_by_category[party] = party_votes.groupby("category")["partyline_correct"].mean().reset_index().to_dict(orient="records")

In [ ]:
import pandas as pd
import plotly.express as px

# build a DataFrame: rows = parties, columns = categories
df = pd.DataFrame({
    party: {item['category']: item['partyline_correct'] for item in entries}
    for party, entries in percentage_partyline_by_category.items()
}).T

# reshape to long form for px.bar
df_long = df.reset_index().melt(
    id_vars='index',
    var_name='Category',
    value_name='Correctness'
)
df_long.rename(columns={'index': 'Party'}, inplace=True)

fig = px.bar(
    df_long,
    x='Party',
    y='Correctness',
    color='Category',
    barmode='group',
    
)

fig.update_layout(
    xaxis_title='Party',
    yaxis_title='Partyline Correctness',
    legend_title_text='Category',
        template='plotly_dark'

)

fig.show()


In [6]:
df.grouoby("category")

AttributeError: 'DataFrame' object has no attribute 'grouoby'